In [ ]:
# ========================
# BENCHMARK TEST SETİ
# Kolaydan Zora Doğru 10 Soru
# ========================

benchmark_questions = [
    {
        "id": 1,
        "difficulty": "Kolay",
        "category": "Basit Veri Çekme",
        "question": "Tesla'nın güncel hisse fiyatı nedir?",
        "expected_tools": ["get_stock_price"],
        "expected_steps": 1
    },
    {
        "id": 2,
        "difficulty": "Kolay",
        "category": "Döviz Bilgisi",
        "question": "1 Dolar kaç TL?",
        "expected_tools": ["get_currency_rate"],
        "expected_steps": 1
    },
    {
        "id": 3,
        "difficulty": "Orta",
        "category": "Finansal Terim + Hesaplama",
        "question": "P/E ratio nedir ve Apple'ın P/E ratio'su sektör ortalamasından kaç puan fazla?",
        "expected_tools": ["search_financial_knowledge", "get_financial_ratios", "calculator"],
        "expected_steps": 3
    },
    {
        "id": 4,
        "difficulty": "Orta",
        "category": "Karşılaştırmalı Analiz",
        "question": "Microsoft ve Apple'ın piyasa değerlerini karşılaştır, hangisi daha büyük?",
        "expected_tools": ["get_stock_info", "get_stock_info"],
        "expected_steps": 2
    },
    {
        "id": 5,
        "difficulty": "Orta",
        "category": "Çoklu Veri + Hesaplama",
        "question": "Amazon'un son kapanış fiyatının %15'i ne kadardır?",
        "expected_tools": ["get_stock_price", "calculator"],
        "expected_steps": 2
    },
    {
        "id": 6,
        "difficulty": "Orta-Zor",
        "category": "Multi-Hop Reasoning",
        "question": "Google'ın borç/özsermaye oranı 1'den büyük mü? Bu ne anlama gelir?",
        "expected_tools": ["get_financial_ratios", "search_financial_knowledge"],
        "expected_steps": 2
    },
    {
        "id": 7,
        "difficulty": "Zor",
        "category": "Yatırım Analizi",
        "question": "NVIDIA hissesi alınır mı? P/E, PEG ve sektör bilgilerini analiz ederek tavsiye ver.",
        "expected_tools": ["get_stock_info", "get_financial_ratios", "search_financial_knowledge"],
        "expected_steps": 4
    },
    {
        "id": 8,
        "difficulty": "Zor",
        "category": "Portföy Hesaplama",
        "question": "10000 TL param var. Bunun %60'ını Apple, %40'ını Microsoft'a yatırsam kaçar TL'lik hisse alırım? (1 USD = 32 TL kabul et)",
        "expected_tools": ["calculator", "calculator", "calculator", "get_stock_price", "get_stock_price"],
        "expected_steps": 5
    },
    {
        "id": 9,
        "difficulty": "Çok Zor",
        "category": "Risk Analizi + Strateji",
        "question": "Bitcoin'e yatırım yapmak istiyorum ama risk toleransım düşük. Stop-loss nedir ve Bitcoin için nasıl bir strateji önerirsin?",
        "expected_tools": ["get_crypto_price", "search_financial_knowledge", "calculator"],
        "expected_steps": 4
    },
    {
        "id": 10,
        "difficulty": "Çok Zor",
        "category": "Kompleks Karşılaştırma",
        "question": "Teknoloji ve finans sektörlerinin özelliklerini karşılaştır. Tesla (teknoloji) ve JPMorgan (finans) hisselerinden hangisi daha riskli? P/E oranlarını ve sektör özelliklerini kullanarak analiz yap.",
        "expected_tools": ["search_financial_knowledge", "get_financial_ratios", "get_financial_ratios", "get_stock_info", "get_stock_info"],
        "expected_steps": 6
    }
]

print(f"✅ {len(benchmark_questions)} benchmark sorusu hazır!")
print("\n📊 Zorluk Dağılımı:")
print(f"   Kolay: 2 soru")
print(f"   Orta: 3 soru")
print(f"   Orta-Zor: 1 soru")
print(f"   Zor: 2 soru")
print(f"   Çok Zor: 2 soru")

In [ ]:
import time
import json
from datetime import datetime

class BenchmarkRunner:
    """Benchmark testlerini çalıştırır ve metrik toplar"""
    
    def __init__(self):
        self.results = []
        
    def run_single_test(self, test_case, verbose=False):
        """Tek bir test case'i çalıştır"""
        
        print(f"\n{'='*70}")
        print(f"🧪 Test #{test_case['id']} - {test_case['difficulty']} - {test_case['category']}")
        print(f"❓ Soru: {test_case['question']}")
        print('='*70)
        
        start_time = time.time()
        
        try:
            # Agent'ı çalıştır
            response = run_agent(
                test_case['question'], 
                max_turns=15,  # Zor sorular için daha fazla tur
                verbose=verbose
            )
            
            elapsed_time = time.time() - start_time
            success = "Answer:" in response or "Cevap:" in response
            
            result = {
                "test_id": test_case['id'],
                "difficulty": test_case['difficulty'],
                "category": test_case['category'],
                "question": test_case['question'],
                "success": success,
                "response": response,
                "elapsed_time": round(elapsed_time, 2),
                "expected_steps": test_case['expected_steps']
            }
            
            self.results.append(result)
            
            status = "✅ BAŞARILI" if success else "❌ BAŞARISIZ"
            print(f"\n{status} - Süre: {elapsed_time:.2f}s")
            
            return result
            
        except Exception as e:
            print(f"\n❌ HATA: {str(e)}")
            elapsed_time = time.time() - start_time
            
            result = {
                "test_id": test_case['id'],
                "difficulty": test_case['difficulty'],
                "category": test_case['category'],
                "question": test_case['question'],
                "success": False,
                "response": f"ERROR: {str(e)}",
                "elapsed_time": round(elapsed_time, 2),
                "expected_steps": test_case['expected_steps']
            }
            
            self.results.append(result)
            return result
    
    def run_all_tests(self, questions, verbose=False):
        """Tüm testleri çalıştır"""
        
        print("\n" + "🚀 BENCHMARK TESTLERİ BAŞLIYOR ".center(70, "="))
        print(f"Toplam Test Sayısı: {len(questions)}")
        print(f"Başlangıç Zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*70 + "\n")
        
        for i, test in enumerate(questions, 1):
            print(f"\n[{i}/{len(questions)}] İşleniyor...")
            self.run_single_test(test, verbose=verbose)
            
            # Her testten sonra kısa bekleme (rate limit için)
            if i < len(questions):
                time.sleep(2)
        
        print("\n" + "✅ TÜM TESTLER TAMAMLANDI ".center(70, "="))
        
        return self.results
    
    def calculate_metrics(self):
        """Metrikleri hesapla"""
        
        if not self.results:
            print("❌ Henüz test sonucu yok!")
            return None
        
        total_tests = len(self.results)
        successful_tests = sum(1 for r in self.results if r['success'])
        failed_tests = total_tests - successful_tests
        
        success_rate = (successful_tests / total_tests) * 100
        avg_time = sum(r['elapsed_time'] for r in self.results) / total_tests
        
        # Zorluk seviyesine göre başarı oranı
        difficulty_stats = {}
        for result in self.results:
            diff = result['difficulty']
            if diff not in difficulty_stats:
                difficulty_stats[diff] = {'total': 0, 'success': 0}
            
            difficulty_stats[diff]['total'] += 1
            if result['success']:
                difficulty_stats[diff]['success'] += 1
        
        metrics = {
            "total_tests": total_tests,
            "successful": successful_tests,
            "failed": failed_tests,
            "success_rate": round(success_rate, 2),
            "avg_time": round(avg_time, 2),
            "difficulty_breakdown": difficulty_stats,
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        return metrics
    
    def print_report(self):
        """Raporu ekrana yazdır"""
        
        metrics = self.calculate_metrics()
        
        if not metrics:
            return
        
        print("\n" + "📊 BENCHMARK RAPORU ".center(70, "="))
        print(f"\n⏰ Tarih: {metrics['timestamp']}")
        print(f"📝 Toplam Test: {metrics['total_tests']}")
        print(f"✅ Başarılı: {metrics['successful']}")
        print(f"❌ Başarısız: {metrics['failed']}")
        print(f"📈 Başarı Oranı: %{metrics['success_rate']}")
        print(f"⚡ Ortalama Süre: {metrics['avg_time']}s")
        
        print(f"\n{'─'*70}")
        print("📊 Zorluk Seviyesine Göre Başarı Oranı:")
        print(f"{'─'*70}")
        
        for diff, stats in metrics['difficulty_breakdown'].items():
            rate = (stats['success'] / stats['total']) * 100
            bar_length = int(rate / 5)  # 20 karakterlik bar
            bar = "█" * bar_length + "░" * (20 - bar_length)
            print(f"{diff:12} | {bar} | {stats['success']}/{stats['total']} (%{rate:.0f})")
        
        print(f"\n{'─'*70}")
        print("📋 Detaylı Test Sonuçları:")
        print(f"{'─'*70}\n")
        
        for result in self.results:
            status = "✅" if result['success'] else "❌"
            print(f"{status} Test #{result['test_id']} ({result['difficulty']}) - {result['elapsed_time']}s")
            print(f"   {result['question'][:60]}...")
            if not result['success']:
                print(f"   ⚠️ Hata: {result['response'][:100]}...")
            print()
        
        print("="*70 + "\n")
        
        return metrics
    
    def save_results(self, filename="benchmark_results.json"):
        """Sonuçları JSON dosyasına kaydet"""
        
        metrics = self.calculate_metrics()
        
        export_data = {
            "metrics": metrics,
            "results": self.results
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        print(f"💾 Sonuçlar '{filename}' dosyasına kaydedildi!")


print("✅ Benchmark runner hazır!")

In [ ]:
# Benchmark runner'ı oluştur
runner = BenchmarkRunner()

# Testleri çalıştır
# verbose=True yaparsanız her adımı görürsünüz (uzun çıktı)
# verbose=False yaparsanız sadece özet görürsünüz (önerilen)

print("⚠️ DİKKAT: Tüm testler yaklaşık 5-10 dakika sürebilir!")
print("⚠️ API rate limit'e takılırsanız testler arası bekleme süresini artırın.\n")

input("Devam etmek için Enter'a basın...")

results = runner.run_all_tests(benchmark_questions, verbose=False)

⚠️ DİKKAT: Tüm testler yaklaşık 5-10 dakika sürebilir!
⚠️ API rate limit'e takılırsanız testler arası bekleme süresini artırın.

Devam etmek için Enter'a basın...

====================🚀 BENCHMARK TESTLERİ BAŞLIYOR ====================
Toplam Test Sayısı: 10
Başlangıç Zamanı: 2025-12-30 17:42:25
======================================================================


[1/10] İşleniyor...

======================================================================
🧪 Test #1 - Kolay - Basit Veri Çekme
❓ Soru: Tesla'nın güncel hisse fiyatı nedir?
======================================================================

✅ BAŞARILI - Süre: 5.25s

[2/10] İşleniyor...

======================================================================
🧪 Test #2 - Kolay - Döviz Bilgisi
❓ Soru: 1 Dolar kaç TL?
======================================================================

✅ BAŞARILI - Süre: 5.10s

[3/10] İşleniyor...

======================================================================
🧪 Test #3 - Orta - Finansal Terim + Hesaplama
❓ Soru: P/E ratio nedir ve Apple'ın P/E ratio'su sektör ortalamasından kaç puan fazla?
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'P/E ratio nedir'

✅ BAŞARILI - Süre: 15.25s

[4/10] İşleniyor...

======================================================================
🧪 Test #4 - Orta - Karşılaştırmalı Analiz
❓ Soru: Microsoft ve Apple'ın piyasa değerlerini karşılaştır, hangisi daha büyük?
======================================================================

✅ BAŞARILI - Süre: 12.48s

[5/10] İşleniyor...

======================================================================
🧪 Test #5 - Orta - Çoklu Veri + Hesaplama
❓ Soru: Amazon'un son kapanış fiyatının %15'i ne kadardır?
======================================================================

✅ BAŞARILI - Süre: 8.79s

[6/10] İşleniyor...

======================================================================
🧪 Test #6 - Orta-Zor - Multi-Hop Reasoning
❓ Soru: Google'ın borç/özsermaye oranı 1'den büyük mü? Bu ne anlama gelir?
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'Debt/Equity ratio yüksek ne demek'

✅ BAŞARILI - Süre: 14.82s

[7/10] İşleniyor...

======================================================================
🧪 Test #7 - Zor - Yatırım Analizi
❓ Soru: NVIDIA hissesi alınır mı? P/E, PEG ve sektör bilgilerini analiz ederek tavsiye ver.
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'P/E ratio 46 teknoloji sektörü yüksek risk'

✅ BAŞARILI - Süre: 21.87s

[8/10] İşleniyor...

======================================================================
🧪 Test #8 - Zor - Portföy Hesaplama
❓ Soru: 10000 TL param var. Bunun %60'ını Apple, %40'ını Microsoft'a yatırsam kaçar TL'lik hisse alırım? (1 USD = 32 TL kabul et)
======================================================================

✅ BAŞARILI - Süre: 29.20s

[9/10] İşleniyor...

======================================================================
🧪 Test #9 - Çok Zor - Risk Analizi + Strateji
❓ Soru: Bitcoin'e yatırım yapmak istiyorum ama risk toleransım düşük. Stop-loss nedir ve Bitcoin için nasıl bir strateji önerirsin?
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'stop-loss nedir nasıl kullanılır'

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'düşük risk toleransı kripto yatırım stratejisi'

✅ BAŞARILI - Süre: 21.64s

[10/10] İşleniyor...

======================================================================
🧪 Test #10 - Çok Zor - Kompleks Karşılaştırma
❓ Soru: Teknoloji ve finans sektörlerinin özelliklerini karşılaştır. Tesla (teknoloji) ve JPMorgan (finans) hisselerinden hangisi daha riskli? P/E oranlarını ve sektör özelliklerini kullanarak analiz yap.
======================================================================

🔍 [RAG TARAMASI] Bilgi bankasından aranıyor: 'teknoloji vs finans sektörü risk karşılaştırması'

✅ BAŞARILI - Süre: 27.10s

======================✅ TÜM TESTLER TAMAMLANDI =======================

In [ ]:
# Benchmark sonuçlarını detaylı göster
print("\n" + "="*70)
print("📝 DETAYLI TEST SONUÇLARI".center(70))
print("="*70 + "\n")

for i, result in enumerate(runner.results, 1):
    status = "✅" if result['success'] else "❌"
    
    print(f"\n{'-'*70}")
    print(f"{status} Test #{result['test_id']} - {result['difficulty']} - {result['category']}")
    print(f"{'-'*70}")
    print(f"❓ Soru: {result['question']}")
    print(f"⏱️  Süre: {result['elapsed_time']}s")
    print(f"\n💬 Cevap:")
    print("-" * 70)
    
    # Answer kısmını ayıkla
    response = result['response']
    
    # Answer veya Cevap kelimelerinden sonrasını al
    if "Answer:" in response:
        answer_text = response.split("Answer:")[-1].strip()
        print(answer_text[:500])  # İlk 500 karakter
    elif "Cevap:" in response:
        answer_text = response.split("Cevap:")[-1].strip()
        print(answer_text[:500])
    else:
        # Tam yanıtı göster
        print(response[:500])
    
    print("\n")

print("="*70)


======================================================================
                       📝 DETAYLI TEST SONUÇLARI                       
======================================================================


----------------------------------------------------------------------
✅ Test #1 - Kolay - Basit Veri Çekme
----------------------------------------------------------------------
❓ Soru: Tesla'nın güncel hisse fiyatı nedir?
⏱️  Süre: 5.25s

💬 Cevap:
----------------------------------------------------------------------
Tesla'nın güncel hisse fiyatı $461.55



----------------------------------------------------------------------
✅ Test #2 - Kolay - Döviz Bilgisi
----------------------------------------------------------------------
❓ Soru: 1 Dolar kaç TL?
⏱️  Süre: 5.1s

💬 Cevap:
----------------------------------------------------------------------
1 Dolar = 42.93 TL



----------------------------------------------------------------------
✅ Test #3 - Orta - Finansal Terim + Hesaplama
----------------------------------------------------------------------
❓ Soru: P/E ratio nedir ve Apple'ın P/E ratio'su sektör ortalamasından kaç puan fazla?
⏱️  Süre: 15.25s

💬 Cevap:
----------------------------------------------------------------------
**P/E Ratio (Fiyat/Kazanç Oranı) Nedir:**
- Hisse senedi fiyatının, şirketin hisse başına kazancına bölünmesiyle bulunur
- Düşük P/E = Hisse ucuz olabilir
- Yüksek P/E = Hisse pahalı veya büyüme beklentisi var
- Sektör ortalaması genellikle 15-20 arasındadır

**Apple'ın Durumu:**
- Apple'ın P/E Ratio'su: 36.52
- Sektör ortalaması: ~17.5 (15-20 arası ortalama)
- **Fark: Yaklaşık 19 puan fazla**

Bu, Apple'ın sektör ortalamasına göre oldukça pahalı değerlendiğini gösteriyor. Yatırımcılar Apple'dan



----------------------------------------------------------------------
✅ Test #4 - Orta - Karşılaştırmalı Analiz
----------------------------------------------------------------------
❓ Soru: Microsoft ve Apple'ın piyasa değerlerini karşılaştır, hangisi daha büyük?
⏱️  Süre: 12.48s

💬 Cevap:
----------------------------------------------------------------------
**Piyasa Değeri Karşılaştırması:**

🍎 **Apple Inc.**
- Piyasa Değeri: $4,048,309,977,088 (~$4.05 trilyon)

🖥️ **Microsoft Corporation**  
- Piyasa Değeri: $3,629,047,873,536 (~$3.63 trilyon)

**Sonuç:** Apple'ın piyasa değeri Microsoft'tan yaklaşık **$419 milyar** daha büyük. Apple şu anda dünyanın en değerli şirketi konumunda.

**Ek Bilgiler:**
- Her iki şirket de teknoloji sektöründe
- Apple: Tüketici elektroniği odaklı
- Microsoft: Yazılım altyapısı odaklı
- Apple daha az çalışanla (166K vs 2



----------------------------------------------------------------------
✅ Test #5 - Orta - Çoklu Veri + Hesaplama
----------------------------------------------------------------------
❓ Soru: Amazon'un son kapanış fiyatının %15'i ne kadardır?
⏱️  Süre: 8.79s

💬 Cevap:
----------------------------------------------------------------------
Amazon'un son kapanış fiyatı $231.35'tir. Bunun %15'i $34.70'tir.



----------------------------------------------------------------------
✅ Test #6 - Orta-Zor - Multi-Hop Reasoning
----------------------------------------------------------------------
❓ Soru: Google'ın borç/özsermaye oranı 1'den büyük mü? Bu ne anlama gelir?
⏱️  Süre: 14.82s

💬 Cevap:
----------------------------------------------------------------------
Evet, Google'ın (GOOGL) borç/özsermaye oranı 11.424 olup 1'den çok büyük.

**Bu ne anlama gelir:**

🔴 **Yüksek Finansal Risk**: 11.424 oranı oldukça yüksek (2'den büyük oranlar riskli kabul edilir)

📊 **Anlamı**: 
- Google'ın borçları, özsermayesinin 11.4 katı
- Şirket ağırlıklı olarak borçla finanse ediliyor
- Faiz oranları yükseldiğinde daha fazla etkilenir

⚠️ **Dikkat**: 
- Teknoloji şirketleri için bu oran sektörel olarak değerlendirilmeli
- Google'ın güçlü nakit akışı bu riski azaltabilir




----------------------------------------------------------------------
✅ Test #7 - Zor - Yatırım Analizi
----------------------------------------------------------------------
❓ Soru: NVIDIA hissesi alınır mı? P/E, PEG ve sektör bilgilerini analiz ederek tavsiye ver.
⏱️  Süre: 21.87s

💬 Cevap:
----------------------------------------------------------------------
NVIDIA (NVDA) Hisse Analizi:

📊 **Finansal Durum:**
- Güncel Fiyat: $187.96
- P/E Ratio: 46.4 (teknoloji sektörü ortalaması 25-40'ın üzerinde)
- PEG Ratio: Mevcut değil
- Borç/Özkaynak: 9.1 (yüksek borçlanma)
- ROE: %107 (çok güçlü karlılık)
- Profit Margin: %53 (mükemmel kar marjı)

⚖️ **Analiz:**

**Güçlü Yönler:**
- AI ve yarı iletken lideri
- Olağanüstü kar marjı (%53)
- Güçlü ROE (%107)
- 4.5 trilyon $ piyasa değeri

**Risk Faktörleri:**
- P/E 46.4 çok yüksek (pahalı değerleme)
- Yüksek bor



----------------------------------------------------------------------
✅ Test #8 - Zor - Portföy Hesaplama
----------------------------------------------------------------------
❓ Soru: 10000 TL param var. Bunun %60'ını Apple, %40'ını Microsoft'a yatırsam kaçar TL'lik hisse alırım? (1 USD = 32 TL kabul et)
⏱️  Süre: 29.2s

💬 Cevap:
----------------------------------------------------------------------
10.000 TL'lik yatırım planınız:

**Apple (AAPL) - %60 (6.000 TL):**
- 6.000 TL = 187.5 USD
- Apple hisse fiyatı: $272.75
- Alabileceğiniz hisse: 0.69 hisse

**Microsoft (MSFT) - %40 (4.000 TL):**
- 4.000 TL = 125 USD  
- Microsoft hisse fiyatı: $488.06
- Alabileceğiniz hisse: 0.26 hisse

⚠️ **Not:** Kesirli hisse alımı her brokerde mümkün olmayabilir. Tam hisse alımı için daha fazla sermaye gerekebilir.



----------------------------------------------------------------------
✅ Test #9 - Çok Zor - Risk Analizi + Strateji
----------------------------------------------------------------------
❓ Soru: Bitcoin'e yatırım yapmak istiyorum ama risk toleransım düşük. Stop-loss nedir ve Bitcoin için nasıl bir strateji önerirsin?
⏱️  Süre: 21.64s

💬 Cevap:
----------------------------------------------------------------------
Bitcoin yatırımı için düşük riskli strateji önerim:

**Stop-Loss Nedir?**
- Zararı sınırlamak için önceden belirlediğiniz fiyat seviyesi
- Bu fiyata ulaşınca otomatik satış yapılır
- Genellikle %5-10 altında konur

**Bitcoin için Strateji (Güncel fiyat: $88,514.74)**

🔴 **Risk Yönetimi:**
- Stop-loss: $79,663 (%10 zarar sınırı)
- Portföyünüzün maksimum %5-10'unu Bitcoin'e yatırın
- Tek seferde değil, kademeli alım yapın (DCA - Dollar Cost Averaging)

📊 **Düşük Risk Stratejisi:**
- Aylık küçük mi



----------------------------------------------------------------------
✅ Test #10 - Çok Zor - Kompleks Karşılaştırma
----------------------------------------------------------------------
❓ Soru: Teknoloji ve finans sektörlerinin özelliklerini karşılaştır. Tesla (teknoloji) ve JPMorgan (finans) hisselerinden hangisi daha riskli? P/E oranlarını ve sektör özelliklerini kullanarak analiz yap.
⏱️  Süre: 27.1s

💬 Cevap:
----------------------------------------------------------------------
**TESLA vs JPMorgan Risk Analizi:**

## 📊 Finansal Oranlar Karşılaştırması:

**Tesla (TSLA):**
- P/E Ratio: 313.47 (Aşırı yüksek - spekülatif)
- ROE: %6.79
- Profit Margin: %5.31
- Debt/Equity: 17.08

**JPMorgan (JPM):**
- P/E Ratio: 15.99 (Makul seviye)
- ROE: %16.44 (Çok güçlü)
- Profit Margin: %34.7 (Mükemmel)
- Debt/Equity: N/A (Bankalar için farklı)

## 🎯 Sektör Özellikleri:

**Teknoloji Sektörü (Tesla):**
- ✅ Yüksek büyüme potansiyeli
- ❌ Yüksek volatilite
- ❌ P/E oranları 25-40 arası norm


======================================================================